# Examples

In this tutorial, we will show different problems that the causalinflation package can solve.

## Causal compatibility problems

The causal compatibility problem aims to answer the question of whether a given probability distribution of outcomes of measurements is compatible with a given causal model. For the following examples, we consider compatibility with a quantum causal model. 

*TODO*: explain more

In [20]:
import causalinflation as cinf
from causalinflation.quantum.InflationSDP import InflationSDP
from causalinflation.InflationProblem import InflationProblem

from examples_utils import feasibility_bisection


#### The PR box

TODO: break the code and introduce the problem

add bisection function hidden from the documentation, maybe in example_utils.py

In [21]:
from causalinflation.useful_distributions import P_PRbox_array

InfProb = InflationProblem( dag={"h1": ["v1", "v2"]},
                            outcomes_per_party=[2, 2],
                            settings_per_party=[2, 2],
                            inflation_level_per_source=[1],
                            names=['A', 'B'])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation(column_specification='npa1')

InfSDP, critical_vis = feasibility_bisection(InfSDP, P_PRbox_array, verbose=0)

print("Critical visibility:", "{:10.4g}".format(critical_vis))
print("Infeasibility certificate: \n", InfSDP.dual_certificate_as_symbols_probs,
                                       "≥", 0)

Critical visibility:     0.7071
Infeasibility certificate: 
 -201487.332*p(00|00) - 201493.109*p(00|01) - 201493.109*p(00|10) + 201492.612*p(00|11) + 201489.909*pA(0|0) + 1.0*pA(0|1) + 201489.909*pB(0|0) + 1.0*pB(0|1) + 41739.738 ≥ 0


#### The 2PR distribution in the bilocal scenario

TODO: break the code and introduce the problem

In [22]:
from causalinflation.useful_distributions import P_2PR_array

InfProb = InflationProblem( dag={"h1": ["v1", "v2"],
                                 "h2": ["v2", "v3"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2],
                            names=['A', 'B', 'C'])

InfSDP = InflationSDP(InfProb, commuting=True, verbose=0)
InfSDP.generate_relaxation(column_specification='npa3', max_monomial_length=2)

InfSDP, critical_vis = feasibility_bisection(InfSDP, P_2PR_array, verbose=1)

print("Critical visibility:", "{:10.4g}".format(critical_vis))
print("Infeasibility certificate: \n", InfSDP.dual_certificate_as_symbols_probs,
                                       "≥", 0)

max(min eigval):  1.393e-09 	visibility = 0 0.5
Critical visibility:       0.75


AttributeError: 'int' object has no attribute 'dual_certificate_as_symbols_probs'

In [ ]:
InfSDP

0

#### The W distribution

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.useful_distributions import P_W_array


InfProb = InflationProblem(dag={"h2": ["v1", "v2"],
                                "h3": ["v1", "v3"],
                                "h1": ["v2", "v3"]},
                           outcomes_per_party=[2, 2, 2],
                           settings_per_party=[1, 1, 1],
                           inflation_level_per_source=[2, 2, 2],
                           names=['A', 'B', 'C'])

InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)
InfSDP.generate_relaxation(column_specification='npa2')

tol_vis = 1e-4
v0, v1 = 0, 1
vm = (v0 + v1)/2

InfSDP.verbose = 0

iteration = 0
certificate = 0
while abs(v1 - v0) >= tol_vis and iteration < 20:
    p = P_W_array(visibility=vm)
    InfSDP.set_distribution(p, use_lpi_constraints=True)
    InfSDP.solve()

    print("max(min eigval):", "{:10.4g}".format(
        InfSDP.primal_objective), "\tvisibility =", iteration, "{:.4g}".format(vm))
    iteration += 1
    if InfSDP.primal_objective >= 0:
        v0 = vm
        vm = (v0 + v1)/2
    elif InfSDP.primal_objective < 0:
        v1 = vm
        vm = (v0 + v1)/2
        certificate = InfSDP.dual_certificate_as_symbols_probs
    if abs(InfSDP.primal_objective) <= 1e-7:
        break

print("Final values and last valid certificate:")
print("max(min eigval):", "{:10.4g}".format(
    InfSDP.primal_objective), "\tvm =", iteration, "{:10.4g}".format(vm))
print(certificate, "≥", 0)


max(min eigval):   0.004188 	visibility = 0 0.5
max(min eigval):   0.001165 	visibility = 1 0.75
max(min eigval):  -0.001274 	visibility = 2 0.875
max(min eigval):  0.0006713 	visibility = 3 0.8125
max(min eigval):  0.0001666 	visibility = 4 0.8438
max(min eigval): -0.0004739 	visibility = 5 0.8594
max(min eigval): -0.0001339 	visibility = 6 0.8516
max(min eigval):  2.187e-05 	visibility = 7 0.8477
max(min eigval): -5.472e-05 	visibility = 8 0.8496
max(min eigval): -1.609e-05 	visibility = 9 0.8486
max(min eigval):  2.972e-06 	visibility = 10 0.8481
max(min eigval): -6.541e-06 	visibility = 11 0.8484
max(min eigval): -1.779e-06 	visibility = 12 0.8483
max(min eigval):  5.974e-07 	visibility = 13 0.8482
Final values and last valid certificate:
max(min eigval):  5.974e-07 	vm = 14     0.8482
145.768*p(000|000)*pA(0|0) + 145.769*p(000|000)*pB(0|0) + 145.757*p(000|000)*pC(0|0) + 921.574*p(000|000) - 122.504*pA(0|0)**2 - 112.285*pA(0|0)*pAB(00|00) - 112.149*pA(0|0)*pAC(00|00) - 553.411*pA(0

## Optimisation problems

Given a function of the probability distribution, we want to find the maximum over the set of probability distributions compatible with the causal model. This type of problem can also be tackled with causalinflation by optimising over the semidefinite relaxation of the problem.

TODO: refine statement

#### The Svetlichny distribution

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.quantum.general_tools import to_numbers, to_name
import numpy as np
import itertools
import sympy as sp

In [ ]:
InfProb = InflationProblem( dag={"h1": ["v1", "v2"],
                                 "h2": ["v1", "v3"],
                                 "h3": ["v2", "v3"]},
                            outcomes_per_party=[2, 2, 2],
                            settings_per_party=[2, 2, 2],
                            inflation_level_per_source=[2, 2, 2],
                            names=['A', 'B', 'C'] )

InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)

InfSDP.build_columns('physical222', max_monomial_length=2)
cols_S2 = InfSDP.generating_monomials
InfSDP.build_columns('local1')
cols_loc1 = InfSDP.generating_monomials

cols_S2_name = [to_name(np.array(m), InfProb.names) for m in cols_S2[1:]]
cols_loc1_name = [to_name(np.array(m), InfProb.names) for m in cols_loc1[1:]]
cols = cols_S2_name
for m in cols_loc1_name:
    if m not in cols_S2_name:
        cols.append(m)
#cols = cols_S2_name.union(cols_loc1_name)
cols_num = [[0]]
for m in cols:
    cols_num.append(to_numbers(m, InfProb.names))

InfSDP.generate_relaxation(column_specification=[[], [0], [1], [2],
                                                 [0, 0], [0, 1], [1, 1],
                                                 [0, 2], [2, 2], [1, 2]])

measurements = InfSDP.measurements 
A0 = sp.S.One - 2*measurements[0][0][0][0]
A1 = sp.S.One - 2*measurements[0][0][1][0]
B0 = sp.S.One - 2*measurements[1][0][0][0]
B1 = sp.S.One - 2*measurements[1][0][1][0]
C0 = sp.S.One - 2*measurements[2][0][0][0]
C1 = sp.S.One - 2*measurements[2][0][1][0]
objective = sp.expand(A1*B0*C0 + A0*B1*C0 + A0*B0*C1 - A1*B1*C1 - 
                      A0*B1*C1 - A1*B0*C1 - A1*B1*C0 + A0*B0*C0 )

InfSDP.set_objective(objective=objective)  # By default it maximizes

InfSDP.solve(interpreter="MOSEKFusion")

print(InfSDP.primal_objective)


5.656854245790322


#### The distance to a particular distribution

TODO: break the code and introduce the problem

In [ ]:
from causalinflation.useful_distributions import P_W_array


In [ ]:
outcomes_per_party = [2, 2, 2]
settings_per_party = [1, 1, 1]
InfProb = InflationProblem( dag={"h1": ["v1", "v2"],
                                 "h2": ["v1", "v3"],
                                 "h3": ["v2", "v3"]},
                            outcomes_per_party=outcomes_per_party,
                            settings_per_party=settings_per_party,
                            inflation_level_per_source=[2, 2, 2],
                            names=['A', 'B', 'C'] )

InfSDP = InflationSDP(InfProb, commuting=False, verbose = 0)
InfSDP.generate_relaxation(column_specification='local1',)

p_W = P_W_array(visibility=1.0)

measurements = InfSDP.measurements  # This reads as measurements[party][inflation_source][setting][outcome]. TODO remove "inflation_source" as the user doesnt care

sum_abc_pW_squared = np.multiply(p_W,p_W).sum()

sum_abc_p_obs_times_p_W = 0
sum_abc_p_obs_squared = 0

for a, b, c in itertools.product(*[range(o) for o in outcomes_per_party]):
    # < A_11 * A_22 * B_11 * B_22 * C_11 * C_22 >
    p_obs_squared = 1
    p_obs_squared *= measurements[0][0][0][a] * measurements[0][3][0][a] if a==0 else (1-measurements[0][0][0][0]) * (1-measurements[0][3][0][0])
    p_obs_squared *= measurements[1][0][0][b] * measurements[1][3][0][b] if b==0 else (1-measurements[1][0][0][0]) * (1-measurements[1][3][0][0])
    p_obs_squared *= measurements[2][0][0][c] * measurements[2][3][0][c] if c==0 else (1-measurements[2][0][0][0]) * (1-measurements[2][3][0][0])

    sum_abc_p_obs_squared += p_obs_squared

    # p_W(a,b,c) * < A_11 * B_11 * C_11 >
    p_obs_times_p_W = p_W[a, b, c, 0, 0, 0]
    p_obs_times_p_W *= measurements[0][0][0][0] if a==0 else (1-measurements[0][0][0][0])
    p_obs_times_p_W *= measurements[1][0][0][0] if b==0 else (1-measurements[1][0][0][0])
    p_obs_times_p_W *= measurements[2][0][0][0] if c==0 else (1-measurements[2][0][0][0])

    sum_abc_p_obs_times_p_W += p_obs_times_p_W


objective = sum_abc_pW_squared + sum_abc_p_obs_squared - 2 * sum_abc_p_obs_times_p_W
objective = sp.expand(objective)

InfSDP.set_objective(objective=objective, direction='min')  # By default it maximizes
InfSDP.solve(interpreter="MOSEKFusion")

print(InfSDP.primal_objective)

0.33333333121984077


## Appendix: How to customize the generating set

In this tutorial we will showcase how to customize the generating set of monomials.

We can use the built-in functions to construct a generating set according to several hierarchies. To showcase this, let us first look at a simple scenario of only two parties, Alice and Bob, sharing a quantum state.

In [ ]:
InfProb = InflationProblem(dag= {"h1": ["v1", "v2"]},
                           outcomes_per_party=[2, 2],
                           settings_per_party=[2, 2],
                           inflation_level_per_source=[1])
InfSDP = InflationSDP(InfProb, commuting=False, verbose=0)

These are all the measurement operators:

In [ ]:
meas = InfSDP.measurements
meas

[[[[A_1_0_0], [A_1_1_0]]], [[[B_1_0_0], [B_1_1_0]]]]

As a reminder, the indices are interpreted as follows: the last two give the measurement setting and measurement output, and the rest represent on which copy of the source the operator is acting. For example, `A_1_1_0` represents the projection onto copy 1 of source 1 with measurement setting $x=1$ and measurement outcome $a=0$.

Now let us look at a few examples of hierarchies. The first is the NPA levels

In [ ]:
InfSDP.build_columns(column_specification='local1')
print(InfSDP.generating_monomials_sym)

[1, A_1_0_0, A_1_1_0, B_1_0_0, B_1_1_0, A_1_0_0*B_1_0_0, A_1_0_0*B_1_1_0, A_1_1_0*B_1_0_0, A_1_1_0*B_1_1_0]
